In [1]:
import tensorflow as tf
import numpy as np
from time import time
import os
print(tf.__version__)

2022-05-02 09:57:51.239976: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


2.3.0


### Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")
    
    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split("-")[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

### Data load & pre-processing function

In [3]:
def load_mnist():
    (train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1)  # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1)  # [N, 28, 28] -> [N, 28, 28, 1]
    
    train_data, test_data = normalize(train_data, test_data)
    
    train_labels = tf.keras.utils.to_categorical(train_labels, 10)  # [N, ] -> [N, 10]
    test_labels = tf.keras.utils.to_categorical(test_labels, 10)  # [N, ] -> [N, 10]
    
    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    
    return train_data, test_data

### Performance function

In [4]:
def loss_function(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits,
                                                                  y_true=labels,
                                                                  from_logits=True))
    return loss

def accuracy_function(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_function(model, images, labels)
    return tape.gradient(loss, model.variables)

### Create model (class version)

In [12]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()
        
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Flatten())
        
        self.model.add(tf.keras.layers.Dense(units=256, use_bias=True, kernel_initializer=weight_init))
        self.model.add(tf.keras.layers.Activation(tf.keras.activations.relu))
        self.model.add(tf.keras.layers.Dense(units=256, use_bias=True, kernel_initializer=weight_init))
        self.model.add(tf.keras.layers.Activation(tf.keras.activations.relu))
        self.model.add(tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init))
    
    def call(self, x, training=None, mask=None):
        x = self.model(x)
        return x

### Define data & hyper-parameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
        shuffle(buffer_size=100000).\
        prefetch(buffer_size=batch_size).\
        batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
        shuffle(buffer_size=100000).\
        prefetch(buffer_size=len(test_x)).\
        batch(len(test_x))

### Define model & optimizer & writer

In [13]:
""" Model """
network = create_model_class(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = "checkpoints"
logs_dir = "logs"

model_dir = "nn_xavier"

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

### Restore checkpoint & start train or test phase

In [15]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint_counter
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))
                
                train_loss = loss_function(network, train_input, train_label)
                train_accuracy = accuracy_function(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_function(network, test_input, test_label)
                
                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)
                
                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + "-{}".format(counter))

# test phase
else:
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:
        test_accuracy = accuracy_function(network, test_input, test_label)
    
    print("test_Accuracy : %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.7484, train_loss: 2.04681373, train_accuracy: 0.4375, test_Accuracy: 0.2852
Epoch: [ 0] [    1/  468] time: 0.8994, train_loss: 1.97713757, train_accuracy: 0.4453, test_Accuracy: 0.4352
Epoch: [ 0] [    2/  468] time: 1.0591, train_loss: 1.87041545, train_accuracy: 0.5859, test_Accuracy: 0.5069
Epoch: [ 0] [    3/  468] time: 1.1960, train_loss: 1.78761983, train_accuracy: 0.5781, test_Accuracy: 0.5676
Epoch: [ 0] [    4/  468] time: 1.3351, train_loss: 1.61511564, train_accuracy: 0.7188, test_Accuracy: 0.6215
Epoch: [ 0] [    5/  468] time: 1.4799, train_loss: 1.54127312, train_accuracy: 0.7500, test_Accuracy: 0.6829
Epoch: [ 0] [    6/  468] time: 1.6294, train_loss: 1.46439672, train_accuracy: 0.7578, test_Accuracy: 0.7213
Epoch: [ 0] [    7/  468] time: 1.7708, train_loss: 1.23949432, train_accuracy: 0.7734, test_Accuracy: 0.7442
Epoch: [ 0] [    8/  468] time: 1.9169,

Epoch: [ 0] [   74/  468] time: 11.1374, train_loss: 0.34008214, train_accuracy: 0.9141, test_Accuracy: 0.9078
Epoch: [ 0] [   75/  468] time: 11.2842, train_loss: 0.31304747, train_accuracy: 0.8906, test_Accuracy: 0.9034
Epoch: [ 0] [   76/  468] time: 11.4273, train_loss: 0.32688507, train_accuracy: 0.9141, test_Accuracy: 0.8994
Epoch: [ 0] [   77/  468] time: 11.5642, train_loss: 0.21561915, train_accuracy: 0.9453, test_Accuracy: 0.9011
Epoch: [ 0] [   78/  468] time: 11.7105, train_loss: 0.28560698, train_accuracy: 0.9375, test_Accuracy: 0.9105
Epoch: [ 0] [   79/  468] time: 11.8441, train_loss: 0.22135340, train_accuracy: 0.9531, test_Accuracy: 0.9167
Epoch: [ 0] [   80/  468] time: 11.9789, train_loss: 0.28921393, train_accuracy: 0.8984, test_Accuracy: 0.9205
Epoch: [ 0] [   81/  468] time: 12.1096, train_loss: 0.35048127, train_accuracy: 0.8828, test_Accuracy: 0.9213
Epoch: [ 0] [   82/  468] time: 12.2521, train_loss: 0.29015478, train_accuracy: 0.9062, test_Accuracy: 0.9195
E

Epoch: [ 0] [  148/  468] time: 21.5160, train_loss: 0.23979157, train_accuracy: 0.9141, test_Accuracy: 0.9362
Epoch: [ 0] [  149/  468] time: 21.6604, train_loss: 0.22430676, train_accuracy: 0.9297, test_Accuracy: 0.9363
Epoch: [ 0] [  150/  468] time: 21.8082, train_loss: 0.29867285, train_accuracy: 0.8828, test_Accuracy: 0.9376
Epoch: [ 0] [  151/  468] time: 21.9472, train_loss: 0.17450739, train_accuracy: 0.9219, test_Accuracy: 0.9383
Epoch: [ 0] [  152/  468] time: 22.0831, train_loss: 0.19861059, train_accuracy: 0.9609, test_Accuracy: 0.9385
Epoch: [ 0] [  153/  468] time: 22.2159, train_loss: 0.23588218, train_accuracy: 0.9531, test_Accuracy: 0.9384
Epoch: [ 0] [  154/  468] time: 22.3558, train_loss: 0.33318111, train_accuracy: 0.9141, test_Accuracy: 0.9388
Epoch: [ 0] [  155/  468] time: 22.4876, train_loss: 0.21509960, train_accuracy: 0.9141, test_Accuracy: 0.9390
Epoch: [ 0] [  156/  468] time: 22.6401, train_loss: 0.28575256, train_accuracy: 0.9297, test_Accuracy: 0.9397
E

Epoch: [ 0] [  222/  468] time: 31.9394, train_loss: 0.09194787, train_accuracy: 0.9766, test_Accuracy: 0.9528
Epoch: [ 0] [  223/  468] time: 32.0801, train_loss: 0.09121445, train_accuracy: 0.9844, test_Accuracy: 0.9498
Epoch: [ 0] [  224/  468] time: 32.2197, train_loss: 0.26088256, train_accuracy: 0.9297, test_Accuracy: 0.9501
Epoch: [ 0] [  225/  468] time: 32.3611, train_loss: 0.12478704, train_accuracy: 0.9531, test_Accuracy: 0.9503
Epoch: [ 0] [  226/  468] time: 32.5127, train_loss: 0.16432197, train_accuracy: 0.9453, test_Accuracy: 0.9505
Epoch: [ 0] [  227/  468] time: 32.6586, train_loss: 0.14691950, train_accuracy: 0.9609, test_Accuracy: 0.9505
Epoch: [ 0] [  228/  468] time: 32.8011, train_loss: 0.11635704, train_accuracy: 0.9609, test_Accuracy: 0.9503
Epoch: [ 0] [  229/  468] time: 32.9515, train_loss: 0.22408794, train_accuracy: 0.9375, test_Accuracy: 0.9505
Epoch: [ 0] [  230/  468] time: 33.0864, train_loss: 0.12453275, train_accuracy: 0.9766, test_Accuracy: 0.9509
E

Epoch: [ 0] [  296/  468] time: 42.3288, train_loss: 0.19526041, train_accuracy: 0.9531, test_Accuracy: 0.9520
Epoch: [ 0] [  297/  468] time: 42.4639, train_loss: 0.12240323, train_accuracy: 0.9609, test_Accuracy: 0.9522
Epoch: [ 0] [  298/  468] time: 42.5991, train_loss: 0.17470390, train_accuracy: 0.9219, test_Accuracy: 0.9533
Epoch: [ 0] [  299/  468] time: 42.7338, train_loss: 0.13799453, train_accuracy: 0.9766, test_Accuracy: 0.9528
Epoch: [ 0] [  300/  468] time: 42.8760, train_loss: 0.08169749, train_accuracy: 0.9844, test_Accuracy: 0.9508
Epoch: [ 0] [  301/  468] time: 43.0254, train_loss: 0.11748227, train_accuracy: 0.9766, test_Accuracy: 0.9497
Epoch: [ 0] [  302/  468] time: 43.1625, train_loss: 0.18991870, train_accuracy: 0.9609, test_Accuracy: 0.9486
Epoch: [ 0] [  303/  468] time: 43.3088, train_loss: 0.14767304, train_accuracy: 0.9609, test_Accuracy: 0.9502
Epoch: [ 0] [  304/  468] time: 43.4453, train_loss: 0.13671103, train_accuracy: 0.9688, test_Accuracy: 0.9513
E

Epoch: [ 0] [  370/  468] time: 52.7747, train_loss: 0.18213910, train_accuracy: 0.9531, test_Accuracy: 0.9597
Epoch: [ 0] [  371/  468] time: 52.9126, train_loss: 0.16996093, train_accuracy: 0.9531, test_Accuracy: 0.9603
Epoch: [ 0] [  372/  468] time: 53.0628, train_loss: 0.06164036, train_accuracy: 0.9844, test_Accuracy: 0.9604
Epoch: [ 0] [  373/  468] time: 53.2088, train_loss: 0.10060697, train_accuracy: 0.9688, test_Accuracy: 0.9606
Epoch: [ 0] [  374/  468] time: 53.3423, train_loss: 0.09819152, train_accuracy: 0.9766, test_Accuracy: 0.9616
Epoch: [ 0] [  375/  468] time: 53.4818, train_loss: 0.09488378, train_accuracy: 0.9844, test_Accuracy: 0.9610
Epoch: [ 0] [  376/  468] time: 53.6278, train_loss: 0.19839987, train_accuracy: 0.9531, test_Accuracy: 0.9594
Epoch: [ 0] [  377/  468] time: 53.7770, train_loss: 0.09427930, train_accuracy: 0.9766, test_Accuracy: 0.9567
Epoch: [ 0] [  378/  468] time: 53.9129, train_loss: 0.15828058, train_accuracy: 0.9375, test_Accuracy: 0.9549
E

Epoch: [ 0] [  444/  468] time: 63.2300, train_loss: 0.12981951, train_accuracy: 0.9453, test_Accuracy: 0.9583
Epoch: [ 0] [  445/  468] time: 63.3623, train_loss: 0.11032768, train_accuracy: 0.9766, test_Accuracy: 0.9586
Epoch: [ 0] [  446/  468] time: 63.4971, train_loss: 0.16015893, train_accuracy: 0.9453, test_Accuracy: 0.9604
Epoch: [ 0] [  447/  468] time: 63.6338, train_loss: 0.10519493, train_accuracy: 0.9609, test_Accuracy: 0.9617
Epoch: [ 0] [  448/  468] time: 63.7726, train_loss: 0.16856669, train_accuracy: 0.9688, test_Accuracy: 0.9642
Epoch: [ 0] [  449/  468] time: 63.9031, train_loss: 0.09169524, train_accuracy: 0.9922, test_Accuracy: 0.9658
Epoch: [ 0] [  450/  468] time: 64.0394, train_loss: 0.11868545, train_accuracy: 0.9688, test_Accuracy: 0.9646
Epoch: [ 0] [  451/  468] time: 64.1762, train_loss: 0.17730582, train_accuracy: 0.9766, test_Accuracy: 0.9626
Epoch: [ 0] [  452/  468] time: 64.3157, train_loss: 0.09575620, train_accuracy: 0.9531, test_Accuracy: 0.9596
E